# this code does not append pipelines without routes (it skips them)

In [1]:
import pygsheets # use 'pip install pygsheets'
import numpy
import datetime

import pandas
import geopandas
#import geopandas.tools import overlay

import shapely

In [2]:
credentials_directory = '/Users/baird/Dropbox/_google-api/'
gc = pygsheets.authorize(client_secret=credentials_directory+'client_secret.json')
spreadsheet = gc.open_by_key('1foPLE6K-uqFlaYgLPAUxzeXfDO5wOOqE7tibNHeqTek')

gas_pipes = spreadsheet.worksheet('title', 'Gas pipelines').get_as_df(start='A2')
oil_pipes = spreadsheet.worksheet('title', 'Oil/NGL pipelines').get_as_df(start='A2')

#gas_pipes.drop('WKTFormat', axis=1, inplace=True) # delete WKTFormat column
#oil_pipes.drop('WKTFormat', axis=1, inplace=True)

# delete columns that aren't the same in the sheets, to concatenate them...
columns_not_in_oil = list(set(gas_pipes.columns)-set(oil_pipes.columns))
columns_not_in_gas = list(set(oil_pipes.columns)-set(gas_pipes.columns))
gas_pipes.drop(columns=columns_not_in_oil, axis=1, inplace=True)
oil_pipes.drop(columns=columns_not_in_gas, axis=1, inplace=True)

# define whether you want just oil, just gas, or both

In [3]:
#fuel_type = 'oil'
#fuel_type = 'gas'
fuel_type = 'oil-and-gas'

if fuel_type=='oil':
    pipes_orig = oil_pipes
    #pipe.drop(column='CapacityBOEd', inplace=True)
elif fuel_type=='gas':
    pipes_orig = gas_pipes
    #pipe.drop('CapacityBcm/y', inplace=True)
elif fuel_type=='oil-and-gas':  
    pipes_orig = pandas.concat([oil_pipes, gas_pipes], ignore_index=True)

pipes_orig.replace('--', numpy.nan, inplace=True)

In [4]:
# clean up column 'Route'
pipes_orig['Route'] = pipes_orig['Route'].str.strip()

# get rid of "N/A" and any empty routes (which would be empty rows)
pipes_orig = pipes_orig[pipes_orig['Route']!='N/A']
pipes_orig = pipes_orig[pipes_orig['Route']!='']
pipes_orig = pipes_orig[pipes_orig['RouteAccuracy']!='no route']

# possible missing route options
missing_route_options = ['Unavailable',
                         'Capacity expansion only',
                         'Bidirectionality upgrade only',
                         'Short route (< 100 km)']

pipes_noroute_df = pipes_orig.copy()[pipes_orig['Route'].isin(missing_route_options)]
pipes_withroute_df = pipes_orig.copy()[~pipes_orig['Route'].isin(missing_route_options)]

In [5]:
def convert_gfit_to_linestring(coord_str, pipeline_name, segment_name):
    '''
    Takes string from GFIT column of coordinates for a single pipeline,
    converts that string into Shapely LineString or MultiLinestring.
    '''
    #print(pipeline_name, segment_name)
    
    if ':' in coord_str and ';' not in coord_str:
        # simple geometry; no branching
        # create nested list of lists, separating on colons        
        coord_list = coord_str.split(':')
        coord_list_tuples = []
        # non-branched pipeline (nested list with one level)
        # convert nested list of lists to list of tuples
        try:
            for element in coord_list:
                element_tuple = (float(element.split(',')[1]), 
                                 float(element.split(',')[0]))
                coord_list_tuples.append(element_tuple)
        except:
            print(f"Exception for {pipeline_name}; {segment_name}; element: {element}") # for db
        route_conv = shapely.geometry.LineString(coord_list_tuples)

    elif ':' in coord_str and ';' in coord_str:
        # create a nested list of lists, separating on semicolons
        coord_list = coord_str.split(';')   
        # create a second level of nesting, separating on colons
        coord_list = [x.split(':') for x in coord_list]
        # branched pipeline (nested list with two levels)
        route_conv_list_all = []
        
        for nested_list in coord_list:
            coord_list_tuples = []
            # process element
            try:
                for element in nested_list:
                    element_tuple = (float(element.split(',')[1]), 
                                     float(element.split(',')[0]))
                    coord_list_tuples.append(element_tuple)
            except:
                print(f"Exception for {pipeline_name}; segment {segment_name}; element: {element}") # for db
            # process coord_list_tuples
            try:
                route_conv_list = shapely.geometry.LineString(coord_list_tuples)
                route_conv_list_all.append(route_conv_list)
            except:
                print(f"Exception for {pipeline_name}; coord_list_tuples: {coord_list_tuples}") # for db
                pass
        
        route_conv = shapely.geometry.MultiLineString(route_conv_list_all)
        
    return route_conv

In [6]:
def convert_all_pipelines(df):
    """
    Apply the conversion function to all pipelines in the dataframe.
    """
    # create geometry column with empty strings
    df['geometry'] = ''
    
    # filter to keep only pipelines with routes
    mask_route = df['Route'].str.contains(',' or ':')
    pipes_with_route = df.loc[mask_route]
    
    for row in pipes_with_route.index:
        route_str = df.at[row, 'Route']
        pipeline_name = df.at[row, 'PipelineName']
        segment_name = df.at[row, 'SegmentName']
        route_str_converted = convert_gfit_to_linestring(route_str, pipeline_name, segment_name)
        df.at[row, 'geometry'] = route_str_converted
    
    return df

In [7]:
# code to create a dataframe with WKT formatted geometry

# (1) copy gas_pipelines, clean up
to_convert = pipes_withroute_df.copy()#gas_pipes.copy()s)]

# (2) convert all pipelines
pipes_wkt_df = convert_all_pipelines(to_convert)
pipes_wkt_df = pipes_wkt_df.reset_index(drop=True)

# (3) store in a GeoDataFrame, attach a projection, transform to a different one
#pipes_wkt_df['geometry'] = pipes_wkt_df['geometry'].apply(shapely.wkt.loads)
pipes_wkt_gdf = geopandas.GeoDataFrame(pipes_wkt_df, geometry=pipes_wkt_df['geometry'])
pipes_wkt_gdf = pipes_wkt_gdf.set_crs('epsg:4326')
pipes_wkt_gdf_4087 = pipes_wkt_gdf.to_crs('epsg:4087')

# save geodataframe as Excel file with just ProjectID, geometry

In [8]:
pipes_wkt_gdf[['ProjectID','geometry']]

,ProjectID,geometry
0,P0001,"LINESTRING (-111.31072 52.66972, -109.13337 51..."
1,P0002,"LINESTRING (-113.34182 53.57294, -112.91198 53..."
2,P0004,"LINESTRING (-101.73614 48.31197, -102.26074 48..."
3,P0005,"LINESTRING (-113.20655 53.71175, -95.09766 45...."
4,P0006,"LINESTRING (-87.51366 41.58361, -90.13183 38.8..."
...,...,...
3381,P4471,"LINESTRING (-5.74881 41.50704, -6.28714 41.49059)"
3382,P4473,"LINESTRING (3.51253 50.35666, 4.14974 50.50188..."
3383,P4474,"LINESTRING (2.86316 42.46146, 2.96204 42.27070)"
3384,P4475,"LINESTRING (2.96204 42.27070, 2.60499 41.73596)"


In [9]:
now_string = datetime.datetime.now().strftime('%Y-%m-%d')
filename = fuel_type+'-routes-wkt-format-'+now_string+'.xlsx'
pipes_wkt_gdf[['ProjectID','geometry']].to_excel(filename, index=False)
print('saved as', filename)

saved as oil-and-gas-routes-wkt-format-2022-11-07.xlsx


In [10]:
now_string = datetime.datetime.now().strftime('%Y-%m-%d')
filename = fuel_type+'-routes-wkt-format-'+now_string+'.csv'
pipes_wkt_gdf[['ProjectID','geometry']].to_csv(filename, index=False)
print('saved as', filename)

saved as oil-and-gas-routes-wkt-format-2022-11-07.csv
